In [11]:
import mne 
import numpy as np 
import pandas as pd  
import random 
import pickle 
import math

from os.path import exists 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [12]:
# Set the parameters
recording_time=30
file_name =['h','s']
h_num=14
s_num=14
iir_params = {'order': 2, 'ftype': 'butter'}


In [13]:
# Random Subjects shuffling
indice=list(range(1,h_num+1))
random.shuffle(indice)
print(indice)


[11, 4, 13, 5, 12, 6, 3, 2, 10, 8, 9, 7, 1, 14]


In [14]:
# Selecting Channels 
def select_channel(raw,req_channel=['Fp1','Fp2']):
    indice=[]
    for channel in req_channel:
        indice.append(raw.ch_names.index(channel))
    new_raw=raw.get_data(picks=indice)
    return new_raw

In [15]:
#NORMALIZE THE SELECTED DATA
def normalize_eeg_data(new_raw):
    scaler=MinMaxScaler()
    transposed_raw=new_raw.T
    normalized_raw=scaler.fit_transform(transposed_raw)
    new_raw=normalized_raw.T
    return new_raw



In [16]:
# Dividing Into Chunks
def division(new_raw,label,sampling_fre=250,trail_time=30):
    trail_length=sampling_fre*trail_time
    t_l_signal=new_raw.shape[1]
    chunks_amount=t_l_signal//trail_length
    new_raw=new_raw[:,0:chunks_amount*trail_length]
    new_raw=np.split(new_raw,chunks_amount,axis=1)
    features=[]
    for i in new_raw:
        features.append((i,label))
    return features


In [17]:
# Making lengths equal for Balanced Data
def Equal_length(pos_class_train,neg_class_train):
    if len(pos_class_train) > len(neg_class_train):
        pos_class_train = pos_class_train[:len(neg_class_train)]
    elif len(pos_class_train) < len(neg_class_train):
        neg_class_train = neg_class_train[:len(pos_class_train)]
    return pos_class_train,neg_class_train


In [18]:
# Combining all and preprocessing the whole data
def preprocess(subject_name,indices,labels,number_of_subjects=14,train_percent=0.7,Band_Pass_Low_Range=0,Band_Pass_High_Range=50):
    limit=math.ceil(train_percent*number_of_subjects)
    k=1
    train_data=[]
    test_data=[]
    for subject_number in indices:
        if exists('./Dataset/{}{}.edf'.format(subject_name,subject_number)):
            file='./Dataset/{}{}.edf'.format(subject_name,subject_number)
            raw=mne.io.read_raw_edf(file,verbose=0,preload=True,eog=None,exclude=(),stim_channel='auto')
            raw=raw.pick_types(eeg=True)
            raw=raw.filter(Band_Pass_Low_Range,Band_Pass_High_Range,iir_params=iir_params,method='iir')
            new_raw=select_channel(raw)
            new_raw=normalize_eeg_data(new_raw)
            chunks=division(new_raw,label=labels)
            if k<=limit:
                train_data+=chunks
            else:
                test_data+=chunks
            k=k+1
    return train_data,test_data



In [19]:
pos_class_train,pos_class_test=preprocess('h',indices=indice,labels=0)
neg_class_train,neg_class_test=preprocess('s',indices=indice,labels=1)
pos_class_train,neg_class_train=Equal_length(pos_class_train,neg_class_train)
pos_class_test,neg_class_test=Equal_length(pos_class_test,neg_class_test)
training_data=pos_class_train+neg_class_train
testing_data=pos_class_test+neg_class_test

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 4 (effective, after forward-backward)
- Cutoff at 50.00 Hz: -6.02 dB

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 4 (effective, after forward-backward)
- Cutoff at 50.00 Hz: -6.02 dB

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 4 (effective, after forward-backward)
- Cutoff at 50.00 Hz: -6.02 dB

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 4 (effective, after forward-backward)
- Cutoff at 50.00 Hz: -6.02 dB

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non

In [20]:
# Using double backslashes
with open('./samples/training.pkl', 'wb') as f:
    pickle.dump(training_data, f)

with open('./samples/testing.pkl', 'wb') as f:
    pickle.dump(testing_data, f)